**This notebook is for demonstration purposes only. The focus is on illustrating the ideas rather than code implementation details.** For pseudocode and well-organized implementations, please refer to the textbook and the AIMA Python repository:
https://github.com/aimacode/aima-python/tree/master

# $h(n)=$ # of misplaced tiles

In [27]:
import heapq
from dataclasses import dataclass
from typing import Tuple, Dict, Optional, List

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle
from IPython.display import HTML, display

# ============================================================
# A* 8-puzzle with step-by-step annotated animation
# ============================================================

# Start/Goal from your image (0 = blank)
initial: Tuple[int, ...] = (7, 2, 4,
                           5, 0, 6,
                           8, 3, 1)

goal: Tuple[int, ...] = (0, 1, 2,
                         3, 4, 5,
                         6, 7, 8)

N = 3

def h1_misplaced(state: Tuple[int, ...], goal: Tuple[int, ...]) -> int:
    # number of misplaced tiles excluding blank(0)
    return sum(1 for i, v in enumerate(state) if v != 0 and v != goal[i])

def idx_to_rc(i: int) -> Tuple[int, int]:
    return divmod(i, N)

def rc_to_idx(r: int, c: int) -> int:
    return r * N + c

# Neighbors with action label (blank move direction)
# Action meaning: where the BLANK moves
def neighbors_with_action(state: Tuple[int, ...]) -> List[Tuple[Tuple[int, ...], str]]:
    z = state.index(0)
    zr, zc = idx_to_rc(z)
    out = []
    for name, dr, dc in [("Up", -1, 0), ("Down", 1, 0), ("Left", 0, -1), ("Right", 0, 1)]:
        rr, cc = zr + dr, zc + dc
        if 0 <= rr < N and 0 <= cc < N:
            j = rc_to_idx(rr, cc)
            lst = list(state)
            lst[z], lst[j] = lst[j], lst[z]
            out.append((tuple(lst), name))
    return out

@dataclass(order=True)
class PQItem:
    f: int
    tie: int
    state: Tuple[int, ...]
    g: int

def astar(start: Tuple[int, ...], goal: Tuple[int, ...], h_func):
    pq: List[PQItem] = []
    tie = 0

    g_score: Dict[Tuple[int, ...], int] = {start: 0}
    came_from: Dict[Tuple[int, ...], Optional[Tuple[int, ...]]] = {start: None}
    came_action: Dict[Tuple[int, ...], Optional[str]] = {start: None}

    heapq.heappush(pq, PQItem(h_func(start, goal), tie, start, 0))  # f=0+h
    tie += 1

    closed = set()
    
    # search_cost=0
    while pq:
        item = heapq.heappop(pq)
        s = item.state
        if s in closed:
            continue
        closed.add(s)

        if s == goal:
            break
        
        for nb, act in neighbors_with_action(s):
            tg = g_score[s] + 1  # each move cost 1
            # search_cost+=1
            if tg < g_score.get(nb, 10**9):
                g_score[nb] = tg
                came_from[nb] = s
                came_action[nb] = act
                f = tg + h_func(nb, goal)
                heapq.heappush(pq, PQItem(f, tie, nb, tg))
                tie += 1
    
    # print(f"search cost={search_cost}")

    if goal not in came_from:
        return None

    # reconstruct path + actions
    states = []
    actions = []
    cur = goal
    while cur is not None:
        states.append(cur)
        actions.append(came_action[cur])
        cur = came_from[cur]
    states.reverse()
    actions.reverse()
    return states, actions

res = astar(initial, goal, h1_misplaced)
if res is None:
    print("No solution found.")
else:
    path_states, path_actions = res
    print("Solved! Moves =", len(path_states) - 1)

def draw_board(ax, state, header_text, note_text):

    ax.clear()
    ax.set_aspect("equal")
    ax.set_xlim(0, N)
    ax.set_ylim(N, 0)
    ax.axis("off")
    ax.figure.patch.set_facecolor("black")
    ax.set_facecolor("black")

    for r in range(N):
        for c in range(N):
            v = state[rc_to_idx(r, c)]
            face = "#222222" if v == 0 else "#0a72a8"
            ax.add_patch(Rectangle((c, r), 1, 1, facecolor=face, edgecolor="#cccccc", linewidth=1.5))
            if v != 0:
                ax.text(c+0.5, r+0.62, str(v), ha="center", va="center",
                        fontsize=28, color="white", fontweight="bold")

    ax.text(
        0.02, 1.10,
        header_text,
        transform=ax.transAxes,
        fontsize=12,
        color="black"
    )

    ax.text(
        0.02, 1.04,
        note_text,
        transform=ax.transAxes,
        fontsize=10,
        color="black"
    )


def animate_astar(path_states: List[Tuple[int, ...]],
                 path_actions: List[Optional[str]],
                 goal: Tuple[int, ...],
                 h_func,
                 heuristic_func,
                 interval_ms: int = 1000):
    fig, ax = plt.subplots(figsize=(7, 5.5))
    # fig.subplots_adjust(right=0.5, top=0.78) 

    g_list = list(range(len(path_states)))  # each step cost 1
    h_list = [h_func(s, goal) for s in path_states]
    f_list = [g_list[i] + h_list[i] for i in range(len(path_states))]

    def update(k):
        s = path_states[k]
        act = path_actions[k]

        if k == 0:
            action_text = "Start"
            step_cost = 0
        else:
            action_text = f"Action: move blank {act}"
            step_cost = 1

        header = (
            f"A* Search (f=g+h, h={heuristic_func}) "
            f"| step {k}/{len(path_states)-1}"
        )

        note = (
            f"Action: {action_text} | "
            f"step cost={step_cost} | "
            f"g={g_list[k]}, h={h_list[k]}, f={f_list[k]}"
        )

        draw_board(ax, s, header, note)


    anim = FuncAnimation(fig, update, frames=len(path_states), interval=interval_ms, repeat=False)
    plt.close(fig)
    display(HTML(anim.to_jshtml()))

if res is not None:
    animate_astar(path_states, path_actions, goal, h1_misplaced, "misplaced tiles", interval_ms=1000)


Solved! Moves = 26


# $h(n)=$ manhattan distance

In [28]:
def h2_manhattan(state: Tuple[int, ...], goal: Tuple[int, ...]) -> int:
    goal_pos = {v: i for i, v in enumerate(goal)}
    return sum(
        abs(divmod(i, 3)[0] - divmod(goal_pos[v], 3)[0]) +
        abs(divmod(i, 3)[1] - divmod(goal_pos[v], 3)[1])
        for i, v in enumerate(state)
        if v != 0
    )

res = astar(initial, goal, h2_manhattan)
if res is None:
    print("No solution found.")
else:
    path_states, path_actions = res
    print("Solved! Moves =", len(path_states) - 1)

if res is not None:
    animate_astar(path_states, path_actions, goal, h2_manhattan, "manhattan distance", interval_ms=1000)

Solved! Moves = 26


### A more fine-grained heuristic function may sharply reduce the search cost!